# Import dependencies

In [1]:
import pandas as pd
# !pip install openpyxl
import openpyxl
from sqlalchemy import create_engine
import sqlite3 as sql
import pandas as pd


### Retrieve data from excel files

In [2]:
excel_file = 'Data_Tables_LGA_Criminal_Incidents_Year_Ending_March_2022.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None)
sheets = all_sheets.keys()

for sheet_name in sheets:
    sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
    sheet.to_csv("%s.csv" % sheet_name, index=False)

Contents
Table 01

Criminal incidents and rate per 100,000 population by police region and local government area - April 2012 to March 2022
Table 02

Criminal incidents and rate per 100,000 population by principal offence, local government area and police service area - April 2012 to March 2022
Table 03

Criminal incidents by principal offence, local government area and postcode or suburb/town - April 2012 to March 2022
Table 04

Criminal incidents by location type and local government area - April 2012 to March 2022
Table 05

Criminal incidents by charge status and local government area - April 2012 to March 2022
Data exclusions
Table 01

excludes criminal incidents where the geographic location is unknown. For further information of these geographic locations please refer to the Explanatory Notes and Glossary on the website.
Tables 02 - 05

excludes criminal incidents at Justice institutions and immigration facilities, Unincorporated Victoria and where the geographic location is unknown. For further information of these geographic locations please refer to the Explanatory Notes and Glossary on the website.
Notes
Recorded crime statistics are based on data extracted by Victoria police on the 18th day after the reference period, and are subject to movement between releases. For more information about how statistics are compiled, refer to the Explanatory notes on the CSA website.
In order to maintain confidentiality, sensitive offence counts for subdivisions 'A10 Homicide and related offences' and 'A30 Sexual offences' with a value of 3 or less are given a value of 2 to calculate totals. Rates are based on populations provided by the Australian Bureau of Statistics (ABS). The most recent year of data was not available from the ABS in time for it to be used to calculate current year rates. The CSA uses estimates created by the Victorian State Government 'Victoria in Futures' report.
This work is licenced under a Creative Commons Attribution 4.0 International License. When reporting this data you must attribute the Crime Statistics Agency (or CSA) as the source.

### Loop thru excel file to convert various worksheets to csv files

In [3]:
#Create list of dataframes from csv files
file_list = []
for i in range(1,7):
    file_list.append(f"Table 0{i}.csv")

df_list = [pd.read_csv(file) for file in file_list]
file_list

['Table 01.csv',
 'Table 02.csv',
 'Table 03.csv',
 'Table 04.csv',
 'Table 05.csv',
 'Table 06.csv']

### Create and clean police_region_df from Table1 which is high level data, starts from Police Region

In [4]:
# create data frame
region_lga_df = pd.read_csv("Table 01.csv")
# region_lga_df

In [5]:
# strip our blanks from Local Government Area(LGA) and drop duplcates to 
# create a datafream that matched Police Regions to LGA
region_lga_df["Local Government Area"] = region_lga_df["Local Government Area"].str.strip()
region_lga_map_df =region_lga_df[["Local Government Area","Police Region"]].drop_duplicates().reset_index(drop=True)
# region_lga_map_df.info()
# region_lga_map_df.head()


### Create and clean LGA_Offence_df_2019_2022 from Table 02 which in more granular in terms of both area and nature of offences 

In [6]:
# Exract rows that are greater than year 2018 (use 4 years data, 2019-2022) 
# and drop column 'Year ending' as the crime tables are created on March (no date details) every year.
lga_offence_2019_2022_df = df_list[1].loc[df_list[1].Year > 2018].drop ('Year ending', axis=1)
# lga_offence_2019_2022_df.info()

In [7]:
# Join Police Region onto the lga_offence_2019_2022_df
region_psa_lga_offence_2019_2022_df=pd.merge(lga_offence_2019_2022_df, region_lga_df [["Police Region","Local Government Area"]],
       on="Local Government Area")

offence_2019_2022_df = region_psa_lga_offence_2019_2022_df.drop_duplicates()
offence_2019_2022_df.head()

,Year,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population",Local Government Area,Police Service Area,Police Region
0,2022,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089,Ballarat,Ballarat,4 Western
10,2022,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128,Ballarat,Ballarat,4 Western
20,2022,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807,Ballarat,Ballarat,4 Western
30,2022,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740,Ballarat,Ballarat,4 Western
40,2022,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137,Ballarat,Ballarat,4 Western


In [8]:
offence_division_summary_df = df_list[5]
#     .drop('Year ending',axis=1)
# offence_division_summary_df.info()

In [9]:
offence_division_summary_df2=pd.merge(offence_division_summary_df, region_lga_df [["Police Region","Local Government Area"]],
       on="Local Government Area")

offence_division_summary_2019_2022_df = offence_division_summary_df2.drop_duplicates()
offence_division_summary_2019_2022_df.head()

,Year,Local Government Area,A Crimes against the person,B Property and deception offences,C Drug offences,D Public order and security offences,E Justice procedures offences,F Other offences,Police Region
0,2019,Alpine,114,161,26.0,48,50,NaN,2 Eastern
10,2020,Alpine,92,220,24.0,47,52,4.0,2 Eastern
20,2021,Alpine,103,138,21.0,37,62,39.0,2 Eastern
30,2022,Alpine,86,149,16.0,33,61,25.0,2 Eastern
40,2019,Ararat,223,376,65.0,81,165,2.0,4 Western


In [10]:
lgas = offence_division_summary_2019_2022_df['Local Government Area'].unique().tolist()
# lgas

###Export to .csv

In [11]:
import json

payload = {}

for lga in offence_division_summary_2019_2022_df['Local Government Area'].unique().tolist():
    df = offence_division_summary_2019_2022_df.loc[offence_division_summary_2019_2022_df['Local Government Area'] == lga].groupby\
        (['Local Government Area','Year']).agg(sum).reset_index().drop('Local Government Area',axis =1).set_index('Year')
    df
    dataStackedBar = {'labels': df.index.tolist(),'datasets':[]}

    colors = ['#cbd4c2ff','#dbebc0ff','#c3b299ff','#815355ff','#523249ff','#000000']
    color = 0
    for col in df.columns:
        dataStackedBar['datasets'].append({
            'label': col,
            'data': df[col].values.tolist(),
            'backgroundColor': colors[color]
        })
        color += 1
    payload[lga] = dataStackedBar

with open('bardata.json','w') as output:
    json.dump(payload,output)

In [12]:
offence_division_summary_2019_2022_df.to_csv('Offence_division_summary_2019_2022.csv',index=0)
crime = pd.read_csv('Offence_division_summary_2019_2022.csv')

In [13]:
offence_2019_2022_df.to_csv('crime_data_2019_2022.csv',index=0)
crime = pd.read_csv('crime_data_2019_2022.csv')

In [14]:
conn = sql.connect('crime.db')
crime.to_sql('crime', conn)

/Users/juliacooper/opt/anaconda3/envs/bootcamp/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


ValueError: Table 'crime' already exists.

In [15]:
all_df = pd.read_sql('SELECT * FROM crime', conn)

In [20]:
# all_df.head()

In [16]:
all_df.head()


,index,Year,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population",Local Government Area,Police Service Area,Police Region
0,0,2022,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089,Ballarat,Ballarat,4 Western
1,1,2022,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128,Ballarat,Ballarat,4 Western
2,2,2022,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807,Ballarat,Ballarat,4 Western
3,3,2022,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740,Ballarat,Ballarat,4 Western
4,4,2022,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137,Ballarat,Ballarat,4 Western


In [17]:
filter_df = all_df.groupby(["Year","Offence Division","Offence Subdivision"])["Incidents Recorded"].sum()
filter_df = filter_df.reset_index()
filter_df.info()
# save groupby offence division , then sum up incidents recorded record ourput to a variable
# pd.concat to filter_df save as json file 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Year                 112 non-null    int64 
 1   Offence Division     112 non-null    object
 2   Offence Subdivision  112 non-null    object
 3   Incidents Recorded   112 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 3.6+ KB


In [18]:
lga_offence_2022_df = df_list[1].loc[df_list[1].Year > 2021].drop ('Year ending', axis=1)

In [19]:
filter_df2 = lga_offence_2022_df.groupby(["Offence Division","Offence Subdivision", "Offence Subgroup"])["Incidents Recorded"].sum()
filter_df2 = filter_df2.reset_index()
#filter_df2


In [21]:

# var allLgas = 

In [22]:
lga_offence_2021_df = df_list[1].loc[df_list[1].Year == 2021].drop ('Year ending', axis=1)


In [23]:
filter_df3 = lga_offence_2021_df.groupby(["Offence Division","Offence Subdivision", "Offence Subgroup"])["Incidents Recorded"].sum()
filter_df3 = filter_df3.reset_index()
filter_df3

,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,182
1,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,7709
2,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,7788
3,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",2124
4,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,14009
...,...,...,...,...
99,F Other offences,F90 Miscellaneous offences,F91 Environmental offences,20
100,F Other offences,F90 Miscellaneous offences,F92 Public health and safety offences,37139
101,F Other offences,F90 Miscellaneous offences,F93 Cruelty to animals,123
102,F Other offences,F90 Miscellaneous offences,F94 Dangerous substance offences,10


In [24]:
import json

#https://stackoverflow.com/questions/59946453/creating-a-flare-json-to-be-used-in-d3-from-pandas-dataframe
#initialise a flare dictionary
flare = {"name": "flare", "children": []}

# iterate through dataframe values
for row in filter_df2.values:
    level0 = row[0]
    level1 = row[1]
    level2 = row[2]
    value = row[3]
    
    # create a dictionary with all the row data
    d = {'name': level0,
          'children': [{'name': level1,
                        'children': [{'name': level2,
                                      'value': value}]}]}
    # initialize key lists
    key0 = []
    key1 = []

    # iterate through first level node names
    for i in flare['children']:
        key0.append(i['name'])

        # iterate through next level node names
        key1 = []
        for _, v in i.items():
            if isinstance(v, list):
                for x in v:
                    key1.append(x['name'])

    # add the full row of data if the root is not in key0
    if level0 not in key0:
        d = {'name': level0,
              'children': [{'name': level1,
                            'children': [{'name': level2,
                                          'value': value}]}]}
        flare['children'].append(d)

    elif level1 not in key1:

        # if the root exists, then append only the next level children

        d = {'name': level1,
              'children': [{'name': level2,
                            'value': value}]}

        flare['children'][key0.index(level0)]['children'].append(d)

    else:

        # if the root exists, then only append the next level children
        
        d = {'name': level2,
             'value': value}

        flare['children'][key0.index(level0)]['children'][key1.index(level1)]['children'].append(d)

# uncomment next three lines to save as json file
# save to some file
with open('flare2.json', 'w') as outfile:
    json.dump(flare, outfile)

print(json.dumps(flare, indent=2))

{
  "name": "flare",
  "children": [
    {
      "name": "A Crimes against the person",
      "children": [
        {
          "name": "A10 Homicide and related offences",
          "children": [
            {
              "name": "A10 Homicide and related offences",
              "value": 174
            }
          ]
        },
        {
          "name": "A20 Assault and related offences",
          "children": [
            {
              "name": "A211 FV Serious assault",
              "value": 7043
            },
            {
              "name": "A212 Non-FV Serious assault",
              "value": 7814
            },
            {
              "name": "A22 Assault police, emergency services or other authorised officer",
              "value": 1938
            },
            {
              "name": "A231 FV Common assault",
              "value": 13933
            },
            {
              "name": "A232 Non-FV Common assault",
              "value": 9797
            }